In [1]:
from dhlib import BitCard, Player, StateExch, State, Card
import numpy as np

%reload_ext autoreload

In [2]:
with open("log.txt","r") as f:
    log = f.read()
f.closed

True

In [3]:
allLog = np.array(log.split("\n"))

states = []
stateExch = []

nRank = {1: None, 2: None, 3: None, 4: None, 5: None}
for line in allLog:
    if("GAME" in line):
        e = line.split(" ")
        gameId = int(e[1])
        player = []
        pRank = nRank
        nRank = {1: None, 2: None, 3: None, 4: None, 5: None}
        bind = False
        revo = False
        
    if((">>" and "DEAL") in line):
        if(line[2:line.find(" ")] not in [p.name for p in player]):
            player.append(Player(line[2:line.find(" ")],BitCard().genBitCardsFromString(line[line.find("DEAL")+4:])))
        #else:
        #    pass #player.hand = player.hand & BitCard().genBitCardsFromString(line[line.find("DEAL")+4:])
    
    if((">>" and "EXCH") in line):
        e = line.split(" ")
        exchNum = int(e[2])
        if(exchNum == -2):
        #大貧民
            exchTo = pRank[1]
            exchFrom = pRank[5]
        elif(exchNum == -1):
        #貧民
            exchTo = pRank[2]
            exchFrom = pRank[4]
        elif(exchNum == 1):
        #富豪
            exchTo = pRank[4]
            exchFrom = pRank[2]
        elif(exchNum == 2):
        #大富豪
            exchTo = pRank[5]
            exchFrom = pRank[1]
    
        stateEx = StateExch(gameId, exchNum, exchTo, exchFrom, player)
        stateExch.append(stateEx)
        
    if("REDY" in line):
        e = line.split(" ")
        order = {int(i):v[0:v.find("=")] for v,i in zip(e,range(6)) }
        order.pop(0)
        nOrder = order[1]
        pPlayer = []
        state = State(gameId,BitCard().genBitCardsFromString(""),
                        dict(order), Player(nOrder),
                        pRank, dict(nRank),
                        list(player), list(pPlayer), None,
                        False,
                        False)
        states.append(state)
        
    if("BIND" in line):
        pass
    
    if("REVO" in line):
        pass
        
    if("TEND" in line):
        r = [k for k,v in nRank.items() if v is not None]
        if len(r)>=4:
            continue
        
        bind = False
        nOrder = lPlayer
        pPlayer = []
        
        while(nOrder in nRank.values()):
            nOrder = state.getOrderNext(Player(name=nOrder))
        state = State(gameId,BitCard().genBitCardsFromString(""),
                        dict(order), Player(nOrder),
                        pRank, dict(nRank),
                        list(player), list(pPlayer), lPlayer,
                        False,
                        revo)
        states.append(state)
        
    if("GEND" in line):
        e = line.split(" ")
        pl = [p for p in player if p.name==e[1]].pop(0)
        nRank[int(e[2])] = pl.name
        lPlayer = nOrder
        
    if("TURN" in line):       
        e = line.split(" ")
        if(len(e)>2):
            pl = [p for p in player if p.name==e[1]].pop()
            #pl.hand = pl.hand & ~BitCard().genBitCardsFromString(e[2])
            pl.used = pl.used | BitCard().genBitCardsFromString(e[2])
            now = BitCard().genBitCardsFromString(e[2])
        
            nOrder = state.getOrderNext(pl)
            lPlayer = pl.name
            
            if(nOrder in [p.name for p in pPlayer]):
                continue
            while(nOrder in nRank.values()):
                nOrder = state.getOrderNext(Player(name=nOrder))
            bc8 = BitCard(BitCard().genBitCardsFromString("s8,h8,d8,c8"))
            bcNot8 = ~BitCard().genBitCardsFromString("s8,h8,d8,c8") & ~BitCard().genBitCardsFromString("JKR")
            if( (bc8.getNumberOfBitCards(now) >= 1) and
                  (BitCard(bcNot8).getNumberOfBitCards(now)<1) ):
                #8切りフラグ
                continue
            
            bcS = state.board
            now = BitCard().genBitCardsFromString(e[2])
            
            if BitCard().isLock(bcS,now):
                #しばり
                bind = True
            if BitCard().getNumberOfBitCards(now) >= 4:
                #革命
                revo = True
                
            state = State(gameId,BitCard().genBitCardsFromString(e[2]),
                        dict(order), Player(nOrder),
                        pRank, dict(nRank),
                        list(player), list(pPlayer), lPlayer,
                        bind,
                        revo)

            states.append(state)
            
        else:
            #パスした場合
            
            pl = ([p for p in player if p.name==e[1]]).pop()
            pPlayer.append(pl)
            
            nOrder = state.getOrderNext(pl)
            if(nOrder in [p.name for p in pPlayer]):
                continue       
            while(nOrder in nRank.values()):
                nOrder = state.getOrderNext(Player(name=nOrder))
                
            r = [k for k,v in nRank.items() if v is not None]
            if( (nOrder not in [p.name for p in pPlayer]) and 
               (len(pPlayer)+len(r) >= len(player)-1) ):
                continue
            state = State(gameId,state.board,
                        dict(order), Player(nOrder),
                        pRank, dict(nRank),
                        list(player), list(pPlayer), lPlayer,
                        bind,
                        revo)

            states.append(state)

In [4]:
for i in range(10):
    pl = states[i].nOrder
    pl_hand = states[i].getActivePlayerHands()
    temp_board = BitCard(states[i].board)
    print(f'player:{pl.name}')
    print(f'pl_hand:{pl_hand}')
    print(f'player:{temp_board.getStringTableFromBitCards()}')

player:C
pl_hand:5069897507811330
player:   3 4 5 6 7 8 9 X J Q K A 2
c| 0 0 0 0 0 0 0 0 0 0 0 0 0
d| 0 0 0 0 0 0 0 0 0 0 0 0 0
h| 0 0 0 0 0 0 0 0 0 0 0 0 0
s| 0 0 0 0 0 0 0 0 0 0 0 0 0
JKR:0
player:B
pl_hand:2251868542012104
player:   3 4 5 6 7 8 9 X J Q K A 2
c| 0 0 0 0 0 0 0 0 0 0 0 0 0
d| 1 0 0 0 0 0 0 0 0 0 0 0 0
h| 0 0 0 0 0 0 0 0 0 0 0 0 0
s| 0 0 0 0 0 0 0 0 0 0 0 0 0
JKR:0
player:A
pl_hand:1275433661284400
player:   3 4 5 6 7 8 9 X J Q K A 2
c| 0 0 0 0 0 0 0 0 0 0 0 0 0
d| 0 0 0 0 0 0 0 0 0 0 0 0 0
h| 0 1 0 0 0 0 0 0 0 0 0 0 0
s| 0 0 0 0 0 0 0 0 0 0 0 0 0
JKR:0
player:D
pl_hand:373989663310080
player:   3 4 5 6 7 8 9 X J Q K A 2
c| 0 0 0 0 0 0 0 0 0 0 0 0 0
d| 0 0 0 0 0 0 0 0 0 0 0 0 0
h| 0 0 0 0 0 0 0 0 0 0 0 0 0
s| 0 0 0 0 0 0 0 0 0 0 1 0 0
JKR:0
player:E
pl_hand:36009880323077
player:   3 4 5 6 7 8 9 X J Q K A 2
c| 0 0 0 0 0 0 0 0 0 0 0 0 1
d| 0 0 0 0 0 0 0 0 0 0 0 0 0
h| 0 0 0 0 0 0 0 0 0 0 0 0 0
s| 0 0 0 0 0 0 0 0 0 0 0 0 0
JKR:0
player:C
pl_hand:5069897507811330
player:  

In [36]:
def bits_to_table(b):
    return BitCard(b).getStringTableFromBitCards()

In [42]:
_0 = 0x0000000000000000 
_1 = 0x1111111111111111 
_2 = 0x2222222222222222 
_3 = 0x3333333333333333
_4 = 0x4444444444444444
_5 = 0x5555555555555555
_6 = 0x6666666666666666
_7 = 0x7777777777777777
_8 = 0x8888888888888888
_9 = 0x9999999999999999
_a = 0xaaaaaaaaaaaaaaaa
_b = 0xbbbbbbbbbbbbbbbb
_c = 0xcccccccccccccccc
_d = 0xdddddddddddddddd
_e = 0xeeeeeeeeeeeeeeee
_f = 0xffffffffffffffff

In [45]:
i = 2
hand = states[i].getActivePlayerHands()
board = states[i].board

def count(c):
    a = (c & _5) + ((c >> 1) & _5)
    return (a & _3) + ((a >> 2) & _3)

def noBindNextQr(qr, rank, qty):
    diff = (( 1 << (qty*4)) - 1) << (rank*4) & _1
    return qr - diff


def legal_hand(b_board, b_hand):
    rank = count(b_hand)
    return rank



print(bits_to_table(board))
print(bits_to_table(legal_hand(board, hand)))
print(bits_to_table(legal_hand(board, hand) & 0x2222222222222222))

   3 4 5 6 7 8 9 X J Q K A 2
c| 0 0 0 0 0 0 0 0 0 0 0 0 0
d| 0 0 0 0 0 0 0 0 0 0 0 0 0
h| 0 1 0 0 0 0 0 0 0 0 0 0 0
s| 0 0 0 0 0 0 0 0 0 0 0 0 0
JKR:0
   3 4 5 6 7 8 9 X J Q K A 2
c| 0 0 0 0 0 0 0 0 0 0 1 1 1
d| 0 1 0 1 0 1 1 0 0 0 0 0 0
h| 0 0 0 0 0 0 0 0 0 0 0 0 0
s| 0 0 0 0 0 0 0 0 0 0 0 0 0
JKR:0
   3 4 5 6 7 8 9 X J Q K A 2
c| 0 0 0 0 0 0 0 0 0 0 0 0 0
d| 0 1 0 1 0 1 1 0 0 0 0 0 0
h| 0 0 0 0 0 0 0 0 0 0 0 0 0
s| 0 0 0 0 0 0 0 0 0 0 0 0 0
JKR:0


In [23]:
for i in range(10):
    print(states[i].board)

0
2
64
8796093022208
281474976710656
281474976710656
4503599627370496
4503599627370496
4503599627370496
0
